#Question 2

This questions is about document classification.

In [1]:
### do not change the code in this cell
# make sure you run this cell

from nltk.tokenize import word_tokenize
from nltk.probability import FreqDist
import nltk
nltk.download("punkt")

positive_reviews=["great product",
                  "excellent value",
                  "performance was excellent",
                  "very very good",
                  "best of its kind"]

negative_reviews=["extremely poor",
                  "would not recommend",
                  "did not work",
                  "very poor product",
                  "not ideal"]

unlabelled_review="great value"


[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Administrator\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


a) Why is stopword removal often applied in document classification tasks? What impact might it have on a classifier trained on the reviews above? (4 marks)

Removing stop words can improve text processing efficiency and increase the attention of the text, which are usually 
important words that pay more attention to the emotion of the text.

b)

i) Tokenize the reviews in the lists `positive_reviews` and `negative_reviews` to produce two new lists called `positive_tokenized` and `negative_tokenized`. For example, the list `["this is a review", "this is another"]` would become `[["this", "is", "a", "review"], ["this", "is", "another"]]`. (6 marks)

In [2]:
def get_tokens(reviews):
    tokens = []
    for phrase in reviews:
        tokens.append(word_tokenize(phrase))
    return tokens    
    
positive_tokenized = get_tokens(positive_reviews)
negative_tokenized = get_tokens(negative_reviews)

print(positive_tokenized)
print(negative_tokenized)

[['great', 'product'], ['excellent', 'value'], ['performance', 'was', 'excellent'], ['very', 'very', 'good'], ['best', 'of', 'its', 'kind']]
[['extremely', 'poor'], ['would', 'not', 'recommend'], ['did', 'not', 'work'], ['very', 'poor', 'product'], ['not', 'ideal']]


ii) Construct a training dataset from the lists of positive and negative tokenized reviews, consisting of a list of two element tuples in which the first element contains the list of tokens and the second element is either "pos" or "neg", indicating the sentiment of the review. For example, the positive review `["a", "good", "review"]` would be represented in the training set as `(["a", "good", "review"], "pos")` (4 marks)

In [4]:
def tag_tokens(tokens, category):
    tokens_tag = []
    for token in tokens:
        if category == "pos":
            tokens_tag.append((token, "pos"))
        elif category == "neg":
            tokens_tag.append((token, "neg"))
    return tokens_tag

positive_tokenized_tag = tag_tokens(positive_tokenized, "pos")
negative_tokenized_tag = tag_tokens(negative_tokenized, "neg")

print(positive_tokenized_tag)
print(negative_tokenized_tag)

[(['great', 'product'], 'pos'), (['excellent', 'value'], 'pos'), (['performance', 'was', 'excellent'], 'pos'), (['very', 'very', 'good'], 'pos'), (['best', 'of', 'its', 'kind'], 'pos')]
[(['extremely', 'poor'], 'neg'), (['would', 'not', 'recommend'], 'neg'), (['did', 'not', 'work'], 'neg'), (['very', 'poor', 'product'], 'neg'), (['not', 'ideal'], 'neg')]


iii) Convert the reviews to a bag-of-words representation using the FreqDist class. For example, the document `["this", "is", "a", "review"]` would become `FreqDist({'this': 1, 'is': 1, 'a': 1, 'review': 1})`. The result should a be a list of pairs, in which one item is the bag-of-words review representation and the other is the sentiment label. (6 marks)

In [6]:
def bag_of_words(tokens_tag):
    bag = []
    for token in tokens_tag:
        bag.append((FreqDist(token[0]), token[1]))
    return bag

pos_bag = bag_of_words(positive_tokenized_tag)
neg_bag = bag_of_words(negative_tokenized_tag)

print(pos_bag)
print(neg_bag)

[(FreqDist({'great': 1, 'product': 1}), 'pos'), (FreqDist({'excellent': 1, 'value': 1}), 'pos'), (FreqDist({'performance': 1, 'was': 1, 'excellent': 1}), 'pos'), (FreqDist({'very': 2, 'good': 1}), 'pos'), (FreqDist({'best': 1, 'of': 1, 'its': 1, 'kind': 1}), 'pos')]
[(FreqDist({'extremely': 1, 'poor': 1}), 'neg'), (FreqDist({'would': 1, 'not': 1, 'recommend': 1}), 'neg'), (FreqDist({'did': 1, 'not': 1, 'work': 1}), 'neg'), (FreqDist({'very': 1, 'poor': 1, 'product': 1}), 'neg'), (FreqDist({'not': 1, 'ideal': 1}), 'neg')]


c)

i) From the training data, calculate the prior probabilities, $p(c)$, of a review being in each sentiment class, $c \in \{ pos , neg \}$. 
$$ p(c) = \frac{freq(c)}{N}$$
Here, $freq(c)$ is the number of documents with label $c$, and $N$ is the total number of documents. The result should be a dictionary whose keys are class labels and whose values are class probabilities. (6 marks)

In [8]:
pos_num = len(positive_reviews)
neg_num = len(negative_reviews)
total_num = pos_num + neg_num

prior_prob_pos = pos_num / total_num
prior_prob_neg = neg_num / total_num

prior_probabilities = {"pos": prior_prob_pos, "neg": prior_prob_neg}

print("Prior Probabilities:")
print(prior_probabilities)

Prior Probabilities:
{'pos': 0.5, 'neg': 0.5}


ii) Calculate the conditional probabilities, $p(f|c)$, of each token feature, $f$, given each sentiment class, $c$.
$$ p(f|c) = \frac{freq(f,c)}{freq(c)}$$ 

Where $freq(f,c)$ is the number of documents in class $c$ containing feature $f$ and $freq(c)$ is the total number of documents in class $c$. The result should be a dictionary of dictionaries, with the outer keys being token features, and the inner keys being class labels whose corresponding values are the conditional probabilities. (10 marks)

In [20]:
def freq_words(tokens):
    freq = {}
    for token in tokens:
        for w in set(token):
            freq[w] = freq.get(w, 0) + 1
    return freq

freq_pos_words = freq_words(positive_tokenized)
freq_neg_words = freq_words(negative_tokenized)

all_unique_words = set(freq_pos_words.keys()).union(freq_neg_words.keys())

print(freq_pos_words)

def calculate_conditional_probabilities():
    conditional_probabilities = {}
    for w in all_unique_words:
        freq_word_given_pos = freq_pos_words.get(w, 0)
        freq_word_given_neg = freq_neg_words.get(w, 0)
        
        prob_word_given_pos = freq_word_given_pos / len(positive_tokenized)
        prob_word_given_neg = freq_word_given_neg / len(negative_tokenized)
    
        word_probabilities = {"pos": prob_word_given_pos, "neg": prob_word_given_neg}
    
        conditional_probabilities[w] = word_probabilities
    return conditional_probabilities

conditional_probabilities = calculate_conditional_probabilities()    
print("Conditional Probabilities:")
print(conditional_probabilities)


{'great': 1, 'product': 1, 'excellent': 2, 'value': 1, 'performance': 1, 'was': 1, 'very': 1, 'good': 1, 'best': 1, 'kind': 1, 'its': 1, 'of': 1}
Conditional Probabilities:
{'would': {'pos': 0.0, 'neg': 0.2}, 'ideal': {'pos': 0.0, 'neg': 0.2}, 'best': {'pos': 0.2, 'neg': 0.0}, 'not': {'pos': 0.0, 'neg': 0.6}, 'great': {'pos': 0.2, 'neg': 0.0}, 'kind': {'pos': 0.2, 'neg': 0.0}, 'value': {'pos': 0.2, 'neg': 0.0}, 'very': {'pos': 0.2, 'neg': 0.2}, 'good': {'pos': 0.2, 'neg': 0.0}, 'poor': {'pos': 0.0, 'neg': 0.4}, 'its': {'pos': 0.2, 'neg': 0.0}, 'of': {'pos': 0.2, 'neg': 0.0}, 'was': {'pos': 0.2, 'neg': 0.0}, 'did': {'pos': 0.0, 'neg': 0.2}, 'recommend': {'pos': 0.0, 'neg': 0.2}, 'performance': {'pos': 0.2, 'neg': 0.0}, 'product': {'pos': 0.2, 'neg': 0.2}, 'extremely': {'pos': 0.0, 'neg': 0.2}, 'work': {'pos': 0.0, 'neg': 0.2}, 'excellent': {'pos': 0.4, 'neg': 0.0}}


iii) Use the priors and conditional probabilities to make a Naive Bayes prediction of the sentiment class for the unlabelled review. (6 marks)

In [24]:
def naive_bayes_predict(review, prior_probabilities, conditional_probabilities):
    tokenized_review = word_tokenize(review)

    prob_pos = prior_probabilities["pos"]
    prob_neg = prior_probabilities["neg"]
    
    smoothing_factor = 0.0
    
    for w in tokenized_review:
        if w in conditional_probabilities:
            prob_pos *= (conditional_probabilities[w]["pos"] + smoothing_factor)
            prob_neg *= (conditional_probabilities[w]["neg"] + smoothing_factor)

    if prob_pos > prob_neg:
        return "pos"
    else:
        return "neg"

predicted_sentiment = naive_bayes_predict(unlabelled_review, prior_probabilities, conditional_probabilities)

print("Predicted Sentiment:", predicted_sentiment)

Predicted Sentiment: pos


d)

i) If you apply the Naive Bayes model to the training data using the class priors and conditional probabilities calculated so far, without smoothing, then you will find that you get predictions of 100% accuracy. What can we say about precision, recall and F1 in this case? (2 marks)

Precision: Precision is the ratio of true positive predictions to the total positive predictions. Since there are no false positives (all predictions are correct), precision would be 100%.

Recall: Recall is the ratio of true positive predictions to the total actual positives. Again, since there are no false negatives (all positive instances are correctly identified), recall would be 100%.

F1 Score: The F1 score is the harmonic mean of precision and recall. Since both precision and recall are 100%, the F1 score would also be 100%.

ii) Should we apply smoothing in this case? Justify your answer. (6 marks)

Reasons to Apply Smoothing:
Avoiding Zero Probabilities

Reasons Not to Apply Smoothing:
In the given scenario where the Naive Bayes model achieves 100% accuracy on the training data, it might be tempting to argue against smoothing since the model appears to perform well without it. 
